In [1]:
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB, GaussianNB

import joblib

In [2]:
class VotingClassifier(object):
    """Stripped-down version of VotingClassifier that uses prefit estimators"""
    def __init__(self, estimators, voting='hard', weights=None):
        self.estimators = [e[1] for e in estimators]
        self.named_estimators = dict(estimators)
        self.voting = voting
        self.weights = weights

    def fit(self, X, y, sample_weight=None):
        raise NotImplementedError
        
    def predict(self, X):
        """ Predict class labels for X.
        Parameters
        ----------
        X : {array-like, sparse matrix}, shape = [n_samples, n_features]
            Training vectors, where n_samples is the number of samples and
            n_features is the number of features.
        Returns
        ----------
        maj : array-like, shape = [n_samples]
            Predicted class labels.
        """

        # check_is_fitted(self, 'estimators')
        if self.voting == 'soft':
            maj = np.argmax(self.predict_proba(X), axis=1)

        else:  # 'hard' voting
            predictions = self._predict(X)
            maj = np.apply_along_axis(lambda x:
                                      np.argmax(np.bincount(x,
                                                weights=self.weights)),
                                      axis=1,
                                      arr=predictions.astype('int'))
        return maj

    def _collect_probas(self, X):
        """Collect results from clf.predict calls. """
        return np.asarray([clf.predict_proba(X) for clf in self.estimators])

    def _predict_proba(self, X):
        """Predict class probabilities for X in 'soft' voting """
        if self.voting == 'hard':
            raise AttributeError("predict_proba is not available when"
                                 " voting=%r" % self.voting)
        #check_is_fitted(self, 'estimators')
        avg = np.average(self._collect_probas(X), axis=0, weights=self.weights)
        return avg

    @property
    def predict_proba(self):
        """Compute probabilities of possible outcomes for samples in X.
        Parameters
        ----------
        X : {array-like, sparse matrix}, shape = [n_samples, n_features]
            Training vectors, where n_samples is the number of samples and
            n_features is the number of features.
        Returns
        ----------
        avg : array-like, shape = [n_samples, n_classes]
            Weighted average probability for each class per sample.
        """
        return self._predict_proba

    def transform(self, X):
        """Return class labels or probabilities for X for each estimator.
        Parameters
        ----------
        X : {array-like, sparse matrix}, shape = [n_samples, n_features]
            Training vectors, where n_samples is the number of samples and
            n_features is the number of features.
        Returns
        -------
        If `voting='soft'`:
          array-like = [n_classifiers, n_samples, n_classes]
            Class probabilities calculated by each classifier.
        If `voting='hard'`:
          array-like = [n_samples, n_classifiers]
            Class labels predicted by each classifier.
        """
        # check_is_fitted(self, 'estimators')
        if self.voting == 'soft':
            return self._collect_probas(X)
        else:
            return self._predict(X)

    def _predict(self, X):
        """Collect results from clf.predict calls. """
        return np.asarray([clf.predict(X) for clf in self.estimators]).T

In [6]:
df = pd.read_csv('spanish_dialects_dataset.csv')
df = df.sample(frac=1).reset_index(drop=True)

vectorizer = TfidfVectorizer (max_features=2000, min_df=3, max_df=0.7)
X = vectorizer.fit_transform(df['Sentences'].values).toarray()

labels_map = {'AR':0, 'CO':1, 'CU':2, 'MX':3, 'PE':4, 'SV':5}
Y = df['Dialect'].map(labels_map).tolist()

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, shuffle=True, random_state=555)

In [8]:
# Random Forest model
rfc = joblib.load('./saved_models/random_forest_classifier.sav')

predictions = rfc.predict(X_test)

print('Random Forest Classifier')
print('Confusion Matrix.\n',confusion_matrix(Y_test,predictions))
print('\nClassification Report.\n',classification_report(Y_test,predictions))
print('\nAccuracy Score.\n',accuracy_score(Y_test, predictions))

c:\users\bhumit\anaconda3\envs\msc\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.22.2.post1 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
c:\users\bhumit\anaconda3\envs\msc\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.22.2.post1 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


Random Forest Classifier
Confusion Matrix.
 [[505  18   5  12  20  12]
 [ 11 516  11   6   7  10]
 [  6  17 380   2  15  12]
 [ 35   9   5 381  13  11]
 [ 11   8   4   5 691   5]
 [ 12  29   7   9   5 508]]

Classification Report.
               precision    recall  f1-score   support

           0       0.87      0.88      0.88       572
           1       0.86      0.92      0.89       561
           2       0.92      0.88      0.90       432
           3       0.92      0.84      0.88       454
           4       0.92      0.95      0.94       724
           5       0.91      0.89      0.90       570

    accuracy                           0.90      3313
   macro avg       0.90      0.89      0.90      3313
weighted avg       0.90      0.90      0.90      3313


Accuracy Score.
 0.8997887111379415


,Sentences,Dialect
2799,pero en la segunda parte argentina ajustó deta...,AR
6164,existen dos jugadas básicas pero hay variantes...,CU
2182,en el otro partido leipzig será local ante man...,AR
12373,c en la carrera pedestre se podrá inscribir ha...,PE
7185,reitero enormes aunke con el perdón de muchos ...,CU
...,...,...
12574,esto será determinado de acuerdo al criterio d...,PE
2512,a yoga artístico individual por género femenin...,AR
13555,foto boca juniors con goles de mauro zárate y ...,PE
2411,una línea derecha de cuadros de un mismo color...,AR
